# Analyse en composantes principales

L'analyse en composantes principales (ACP ou PCA en anglais pour principal component analysis) est une méthode de la famille de l'analyse des données et plus généralement de la statistique multivariée qui consiste à transformer des variables liées entre elles (dites « corrélées ») en nouvelles variables décorrélées les unes des autres. Ces nouvelles variables sont nommées « composantes principales », ou axes principaux. Elle permet de réduire le nombre de variables et de rendre l'information moins redondante. 

Il s'agit d'une approche :
- géométrique : les variables étant représentées dans un nouvel espace, selon des directions d'inertie maximale
- statistique : la recherche portant sur des axes indépendants expliquant au mieux la variabilité — la variance — des données). Lorsqu'on veut compresser un ensemble de $N$ variables aléatoires, les $n$ premiers axes de l'analyse en composantes principales sont un meilleur choix, du point de vue de l'inertie ou de la variance.

L'outil mathématique est appliqué dans d'autres domaines que les statistiques et est parfois appelé décomposition orthogonale aux valeurs propres ou POD (anglais : proper orthogonal decomposition). 

Cours :
http://josephsalmon.eu/enseignement/Montpellier/HMMA308/PCA_slides.pdf

## 1er exemple : analyse des températures

In [ ]:
import pandas as pd

In [ ]:
url = "http://factominer.free.fr/livre/temperat.csv"

In [ ]:
data = pd.read_csv(url,encoding = "ISO-8859-1",sep=";")

In [ ]:
data = pd.DataFrame(data)
... #afficher un résumé des données

On corrige le nom de la première colonne :

In [ ]:
noms_colonnes = ...

In [ ]:
noms_colonnes[...]='Ville'

In [ ]:
data.columns = noms_colonnes

Nombre de variables étudiées :

In [ ]:
...

Nombre d'individus de la population étudiée :

In [ ]:
...

On se contente des données de températures mensuelles :

In [ ]:
temp = ...

In [ ]:
temp

Afficher l'ensemble des températures de la ville de Paris :

In [ ]:
temp.iloc[data[data.Ville == 'Paris'].index ]

On calcule les moyennes par colonne :

In [ ]:
moyennes = ...

In [ ]:
moyennes

On centre les données :

In [ ]:
temp_centrees = ...

Que peut-on dire de la température à Paris (16ème ville) en janvier et en août par rapport à la moyenne ?

In [ ]:
...

La température à Paris est supérieure à la moyenne en janvier mais pas en août.

On calcule les écart types par colonne :

In [ ]:
temp.std(ddof=0)

On réduit les données :

In [ ]:
temp_centreesreduites = temp_centrees/temp.std(ddof=0)

In [ ]:
temp_centreesreduites

Nous pouvons désormais réaliser une Analyse en Composantes Principales sur des données centrées ou centrées réduites (normées). 

Le tableau peut-être analysé à travers ses lignes (les individus) ou les colonnes (les variables). 
- Typologie des individus : il existe une variabilité de températures entre les villes => on veut former des groupes de villes semblables ;
- Typologie des variables : il existe des variables corrélées => on veut former des groupes de variables corrélées.

**Quels groupes de variables expliquent le mieux la variabilité entre les individus ?**

### Nuage des individus
Un individu (une ville) est un point de $\mathbb{R}^{12}$ (espace à 12 dimensions). On cherche à identifier les groupes de points proches et les points isolés. Le nuage de points peut avoir une certaine forme, des directions d'allongement en particulier. Il faut définir une **distance** entre les points.

On choisit une métrique euclidienne : la distance (au carré) entre deux individus $i$ et $j$ est 
$$d^2(i,j) = \sum_{k=1}^{12} (x_{i,k}-x_{j,k})^2$$

Laquelle est de ces deux villes (Athènes et Paris) est la plus éloignée d'Amsterdam ?

In [ ]:
sum((temp_centreesreduites.iloc[0,:]-temp_centreesreduites.iloc[16,:])**2) #distance avec Paris

In [ ]:
... #distance avec Amsterdam

### Nuage des variables
Chaque colonne (un mois) est un vecteur de $\mathbb{R}^{35}$. Le nuage contient $12$ vecteurs et chaque axe de sa représentation correspond à une ville.

On choisit de même une norme euclidienne et le produit scalaire usuel. Ainsi, le produit scalaire entre deux colonnes $i$ et $j$ est :
$$<X_i,X_j> = \sum_{k=1}^{35} x_{k,i}x_{k,j}$$

Du fait que chaque valeur $x_{k,i}$ est une donnée centrée et réduite, on reconnait la définition du coefficient de corrélation linéaire :
$$cor(X_i,X_j) = \frac{1}{n} <X_i,X_j>$$

https://fr.wikipedia.org/wiki/Corr%C3%A9lation_(statistiques)#Coefficient_de_corr%C3%A9lation_lin%C3%A9aire_de_Bravais-Pearson

Du fait que les valeurs du tableau sont réduites, la norme de chaque colonne est égale à 1. On peut donc visualiser l'ensemble des 12 vecteurs sur la sphère unité de $\mathbb{R}^{35}$.

L'angle formé entre deux vecteurs renseigne la corrélation entre les deux variables correspondantes.

### Matrice de corrélation et inertie

On pose $M$ la matrice des variables centrées et réduites. Alors $M^T \cdot M$ est la matrice de corrélation.

In [ ]:
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
M = np.array(temp_centreesreduites) #matrice des données centrées réduites

In [ ]:
corr = np.dot(M.transpose(),M)

La matrice de corrélation est symétrique réelle donc diagonalisable dans une base orthonormée.

D'autre part, on pose $X$ la matrice des variables centrées (non réduites). Alors $\frac{1}{35}X^T \cdot X$ est la matrice de covariance.

In [ ]:
X = np.array(temp_centrees)  #matrice des données centrées
var = np.dot(X.transpose(),X)/35

L'inertie totale est la trace (somme des éléments diagonaux) de la matrice de covariance :

In [ ]:
inertie = np.trace(var)
print(inertie)

On peut vérifier que l'inertie totale est aussi la somme des variances de chaque colonne :

In [ ]:
temp_centrees.var(ddof=0)

In [ ]:
sum(temp_centrees.var(ddof=0))

In [ ]:
sn.heatmap(var, annot=False, fmt='g')
plt.show()

On cherche maintenant les vecteurs propres et les valeurs propres de la matrice de covariance :

In [ ]:
W,v = np.linalg.eig(var)

In [ ]:
print('Valeurs propres : ' + str(W))

In [ ]:
print('Somme des valeurs propres : ' + str(sum(W)))

On retrouve bien l'inertie totale.

Voici la distribution des valeurs propres :

In [ ]:
plt.plot(W,'r+')
plt.show()

La plus grande valeur propre est :

In [ ]:
max(W)

In [ ]:
np.argmax(W)

Le vecteur propre correspondant est :

In [ ]:
v[:,np.argmax(W)]

Le rapport d'inertie est :

In [ ]:
max(W)/inertie

On cherche la deuxième plus grande valeur propre :

In [ ]:
ind = np.argsort(W)[[-1,-2]]

Les deux vecteurs propres correspondant aux deux plus grandes valeurs propres :

In [ ]:
planfactoriel = v[:,ind]

La contribution de ces deux axes à l'inertie totale :

In [ ]:
sum(W[ind])/inertie

### Représentation des individus dans le plan des nouveaux axes

On peut calculer les coordonnées de chaque individu dans le ce plan en projettant orthogonalement sur chaque vecteur du plan :

On calcule par exemple les coordonnées de Paris dans ce plan :

In [ ]:
paris = np.array(tableau.iloc[16,:])

Première coordonnée :

In [ ]:
np.dot(planfactoriel[:,0],paris)

Deuxième coordonnée :

In [ ]:
np.dot(planfactoriel[:,1],paris)

In [ ]:
points = []
for i in temp_centrees.index:
    x = np.dot(planfactoriel[:,0],np.array(tableau.iloc[i,:]))
    y = np.dot(planfactoriel[:,1],np.array(tableau.iloc[i,:]))
    points.append((x,y))
    plt.scatter(x,y,color='blue')
plt.show()

La **qualité** de cette représentation en deux dimension est le rapport d'inertie :

In [ ]:
sum(W[ind])/inertie

### Contribution d'un individu sur un axe

On calcule la coordonnée au carré de l'individu sur l'axe $k$ divisée par $n \lambda_k$.

Par exemple, la contribution de Paris sur le premier axe :

In [ ]:
x,y = points[16]
contrib_1 = x**2/(35*max(W))
print(contrib_1)

### Représentation des variables

On veut voir comment les anciennes variables sont liées aux nouvelles : ici, on rappelle que les variables sont les mois. 

In [ ]:
np.correlate(planfactoriel[:,0],X[:,0])

In [ ]:
planfactoriel[:,0]

In [ ]:
X[:,0]

In [ ]:
len(planfactoriel[:,0])

## Exercice :
Faire les calculs présentés dans cet exemple :

http://www.math.u-bordeaux.fr/~mchave100p/wordpress/wp-content/uploads/2013/10/Exemple_interpret_ACP.pdf

## Pour aller plus loin :
Utiliser la librairire sklearn de Python.

https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

Autre exemple d'analyse des vidéos YouTube :
https://www.kaggle.com/manhari/projet-acp-sur-youtube